In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import logging
from src.utils import setup_logging
setup_logging(console=True, file=False, debug=True)
logger = logging.getLogger(__name__)

[INFO]: Logging setup completed at 08-08-2024-234532        (utils.py:62 [23:45:32])


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import torch.nn.functional as F

from src.modules.sentence_encoders import SentenceKnowledgeEncoder

/home/w/IML/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
import pandas as pd
from data.tempdata import TempData
import matplotlib.pyplot as plt
# Load the data
data_path = '../data/data_with_desc.csv'
data = pd.read_csv(data_path, header=None)
data = TempData(data=data, max_num_context=10, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

temporary_batch_size = 37
batch = data.generate_batch(batch_size=temporary_batch_size, split='train', return_knowledge=True)


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
sentence_encoder = SentenceKnowledgeEncoder(knowledge_dim=128,
                                             freeze_llm=None,
                                             tune_llm_layer_norms=None,
                                             use_bias=True,
                                             device=DEVICE,
                                             knowledge_projection_n_h_layers=1,
                                             knowledge_projection_hidden_dim=128,
                                             knowledge_projection_activation=nn.GELU()).to(DEVICE)




[DEBUG]: https://huggingface.co:443 "HEAD /sentence-transformers/LaBSE/resolve/main/tokenizer_config.json HTTP/11" 200 0        (connectionpool.py:546 [00:04:01])
[DEBUG]: https://huggingface.co:443 "HEAD /sentence-transformers/LaBSE/resolve/main/config.json HTTP/11" 200 0        (connectionpool.py:546 [00:04:01])
[DEBUG]: https://huggingface.co:443 "HEAD /sentence-transformers/LaBSE/resolve/main/model.safetensors HTTP/11" 404 0        (connectionpool.py:546 [00:04:02])


NameError: name 'setence_encoder' is not defined

In [21]:
print(sentence_encoder)

SentenceKnowledgeEncoder(
  (text_encoder): SentenceEncoder(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(501153, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_feat

In [23]:
# print(batch.knowledge)
with torch.no_grad():
    print(sentence_encoder(batch.knowledge).shape)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.4791,  0.5927,  0.9248,  ..., -1.5076, -0.1016, -0.4433],
         [-0.4401,  0.6502,  1.8782,  ..., -1.3402, -0.4429, -0.6051],
         [-0.3522,  0.7034,  1.8348,  ..., -1.5568, -0.4753, -0.4691],
         ...,
         [-0.6987,  0.8121,  1.1449,  ..., -1.2491, -0.2897, -0.2985],
         [-0.7248,  0.6574,  1.1109,  ..., -1.2263, -0.3909, -0.3180],
         [-0.6903,  0.6693,  1.1193,  ..., -1.0519, -0.4405, -0.3304]],

        [[-1.5497,  0.9132,  1.3573,  ..., -2.7220, -0.1757, -1.1877],
         [-0.6321,  0.9146,  1.5150,  ..., -1.7661, -0.1607, -0.8228],
         [-1.0225,  0.4083,  1.1304,  ..., -2.1922, -0.1173, -0.5343],
         ...,
         [-0.8447,  1.0993,  1.7405,  ..., -2.1277, -0.4966, -0.9558],
         [-0.8412,  1.0438,  1.6515,  ..., -2.2248, -0.5137, -0.9991],
         [-0.8565,  1.0592,  1.7573,  ..., -2.0823, -0.4973, -1.0083]],

        [[-0.5362,  0.6886,  0.8796,  ..., -1.4146,  

In [4]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 6.7657e-02,  6.3496e-02,  4.8713e-02,  7.9305e-02,  3.7448e-02,
          2.6528e-03,  3.9375e-02, -7.0985e-03,  5.9361e-02,  3.1537e-02,
          6.0098e-02, -5.2905e-02,  4.0607e-02, -2.5931e-02,  2.9843e-02,
          1.1269e-03,  7.3515e-02, -5.0382e-02, -1.2239e-01,  2.3703e-02,
          2.9727e-02,  4.2477e-02,  2.5634e-02,  1.9952e-03, -5.6919e-02,
         -2.7160e-02, -3.2904e-02,  6.6025e-02,  1.1901e-01, -4.5879e-02,
         -7.2621e-02, -3.2584e-02,  5.2341e-02,  4.5055e-02,  8.2530e-03,
          3.6702e-02, -1.3942e-02,  6.5392e-02, -2.6427e-02,  2.0637e-04,
         -1.3664e-02, -3.6281e-02, -1.9504e-02, -2.8974e-02,  3.9427e-02,
         -8.8409e-02,  2.6243e-03,  1.3671e-02,  4.8306e-02, -3.1157e-02,
         -1.1733e-01, -5.1169e-02, -8.8529e-02, -2.1896e-02,  1.4299e-02,
          4.4417e-02, -1.3482e-02,  7.4339e-02,  2.6638e-02, -1.9876e-02,
          1.7919e-02, -1.0605e-02, -9.0426e-02,  2.1327e-02,  1.4120e-01,
         -6.4718e